![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [130]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [131]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

In [132]:
# Display basic information about the dataset
ds_jobs_transformed.info()

# Display summary statistics of the dataset
ds_jobs_transformed.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
count,19158.000000,19158,19158.000000,14650,19158,18772,18698,16345,19093,13220,13018,18735,19158.000000,19158.000000
unique,NaN,123,NaN,3,2,3,5,6,22,8,6,6,NaN,NaN
top,NaN,city_103,NaN,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1,NaN,NaN
freq,NaN,4355,NaN,13221,13792,13817,11598,14492,3286,3083,9817,8040,NaN,NaN
mean,16875.358179,NaN,0.828848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.366896,0.249348
std,9616.292592,NaN,0.123362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.058462,0.432647
min,1.000000,NaN,0.448000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000
25%,8554.250000,NaN,0.740000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.000000,0.000000
50%,16982.500000,NaN,0.903000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.000000,0.000000
75%,25169.750000,NaN,0.920000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.000000,0.000000


In [133]:
from pandas.api.types import CategoricalDtype

# Convert columns to appropriate data types
ds_jobs_transformed = ds_jobs_transformed.astype({
    'student_id': 'int32',
    'city_development_index': 'float16',
    'training_hours': 'int32'
})

# Convert nominal categories to 'category' type
nominal_columns = ['city', 'gender', 'education_level', 
                   'major_discipline', 'experience', 
                   'company_size', 'company_type', 'last_new_job']

for col in nominal_columns:
    ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')

# Convert 'relevant_experience' column to 'bool' type
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].map({'Has relevant experience': True, 'No relevant experience': False})

# Convert 'job_change' column to 'bool' type
ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].map({0.0: False, 1.0: True})

# Convert 'enrolled_university' column to an ordered category
enrolled_university_order = ['no_enrollment', 'Part time course', 'Full time course']
enrolled_university_dtype = CategoricalDtype(categories=enrolled_university_order, ordered=True)
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype(enrolled_university_dtype)

In [134]:
# Define the order for ordinal categories
education_level_order = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd']
experience_order = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']
company_size_order = ['<10', '10-49', '50-99', '100-500', '500-999', '1000-4999', '5000-9999', '10000+']
last_new_job_order = ['never', '1', '2', '3', '4', '>4']

# Convert columns to ordered categories
ds_jobs_transformed['education_level'] = pd.Categorical(ds_jobs_transformed['education_level'], categories=education_level_order, ordered=True)
ds_jobs_transformed['experience'] = pd.Categorical(ds_jobs_transformed['experience'], categories=experience_order, ordered=True)
ds_jobs_transformed['company_size'] = pd.Categorical(ds_jobs_transformed['company_size'], categories=company_size_order, ordered=True)
ds_jobs_transformed['last_new_job'] = pd.Categorical(ds_jobs_transformed['last_new_job'], categories=last_new_job_order, ordered=True)

In [135]:
# Filter the DataFrame to only contain students with 10 or more years of experience at companies with at least 1000 employees
ds_jobs_transformed = ds_jobs_transformed[
    (ds_jobs_transformed['experience'] >= '10') &
    (ds_jobs_transformed['company_size'] >= '1000-4999')
]

filtered_ds_jobs.value_counts()

student_id  city      city_development_index  gender  relevant_experience  enrolled_university  education_level  major_discipline  experience  company_size  company_type  last_new_job  training_hours  job_change
2           city_103  0.919922                Male    False                no_enrollment        Phd              STEM              15          10000+        NGO           3             128             True          1
23322       city_103  0.919922                Male    True                 no_enrollment        Graduate         STEM              15          10000+        Pvt Ltd       >4            31              True          1
23937       city_73   0.753906                Male    True                 no_enrollment        Graduate         STEM              11          5000-9999     Pvt Ltd       1             90              False         1
23932       city_103  0.919922                Female  True                 no_enrollment        Graduate         STEM              12    

In [136]:
# Check memory usage before and after transformation
memory_usage_before = ds_jobs.info()
memory_usage_after = ds_jobs_transformed.info()

# Display memory usage
memory_usage_before, memory_usage_after

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

(None, None)